In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
testFile = pd.read_csv('/kaggle/input/englishdatasetbig/ASL_Citizen/splits/test.csv')
testFile.head()

In [ ]:
valFile = pd.read_csv('/kaggle/input/englishdatasetbig/ASL_Citizen/splits/val.csv')
valFile.head()

In [ ]:
trainFile = pd.read_csv('/kaggle/input/englishdatasetbig/ASL_Citizen/splits/train.csv')
trainFile.head()

In [ ]:
words = []
mapOfTrain = {}
# mapOfTest ={}
mapOfVal = {}
for word in trainFile['Gloss'].values:
    if len(mapOfTrain)<100:
        words.append(word)
        mapOfTrain[word]=[]
        mapOfVal[word] =[]
        for i in trainFile[trainFile['Gloss']==word]['Video file'].values:
            if len(mapOfTrain[word])<25:
                mapOfTrain[word].append(i)
        for i in (testFile[testFile['Gloss']==word]['Video file'].values):
            if len(mapOfTrain[word])<25:
                mapOfTrain[word].append(i)
        for i in valFile[valFile['Gloss']==word]['Video file'].values:
            mapOfVal[word].append(i)
    else:
        break
for k,v in mapOfTrain.items():
    if len(v) <25:
        print(k,'f'*10)
    print('tr',k,len(v))
#     print('ts',k,len(mapOfTest[k]))
    print('val',k,len(mapOfVal[k]))

In [ ]:
# newVidsMap = {}
# for word in words:
#     newVidsMap[word] = vidsMap[word]
# len(newVidsMap),newVidsMap[words[0]][0]

In [ ]:
!pip install remotezip tqdm opencv-python opencv-python-headless tf-models-official


In [ ]:
!pip install mediapy


In [ ]:
import tqdm
import random
import pathlib
import itertools
import collections
from pathlib import Path
%matplotlib inline
import os
import cv2
import numpy as np
import remotezip as rz
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
# Some modules to display an animation using imageio.
import imageio
from IPython import display
from urllib import request
#from tensorflow_docs.vis import embed
import keras
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
import mediapy as media


# Import the MoViNet model from TensorFlow Models (tf-models-official) for the MoViNet model
from official.projects.movinet.modeling import movinet
from official.projects.movinet.modeling import movinet_model

In [ ]:
print('Num classes:', len(mapOfTrain))
print('Num videos for class[0]:', len(mapOfTrain[words[-1]]))

In [ ]:
NUM_CLASSES = 100
FILES_PER_CLASS = 25

In [ ]:
def select_subset_of_classes(files_for_class, classes, files_per_class):
  """ Create a dictionary with the class name and a subset of the files in that class.

    Args:
      files_for_class: Dictionary of class names (key) and files (values).
      classes: List of classes.
      files_per_class: Number of files per class of interest.

    Returns:
      Dictionary with class as key and list of specified number of video files in that class.
  """
  files_subset = dict()

  for class_name in classes:
    class_files = files_for_class[class_name]
    files_subset[class_name] = class_files[:files_per_class]

  return files_subset


In [ ]:
files_subset = select_subset_of_classes(mapOfTrain, words[:NUM_CLASSES], FILES_PER_CLASS)
files_subset_test = select_subset_of_classes(mapOfVal, words[:NUM_CLASSES], 3)

list(files_subset.keys())

In [ ]:
len(files_subset[words[0]])

In [ ]:
def split_class_lists(files_for_class, count):
  """ Returns the list of files belonging to a subset of data as well as the remainder of
    files that need to be downloaded.

    Args:
      files_for_class: Files belonging to a particular class of data.
      count: Number of files to download.

    Returns:
      Files belonging to the subset of data and dictionary of the remainder of files that need to be downloaded.
  """
  split_files = []
  remainder = {}
  for cls in files_for_class.keys():
    split_files.extend(files_for_class[cls][:count])
    remainder[cls] = files_for_class[cls][count:]
  return split_files, remainder

In [ ]:
import os
import shutil
import tqdm
import pathlib

def read_from_dir(dir_path, to_dir, file_names):
  """ Read the contents of the files from the directory path.

    Args:
      dir_path: A path to a local directory containing data.
      to_dir: A directory to copy data to.
      file_names: Names of files to copy.
  """
  for fn in tqdm.tqdm(file_names):
    class_name = fn.split('/')[-1]
    file_path = fn.split(f'/{class_name}')[0]
    output_dir = os.path.join(to_dir, class_name)
    os.makedirs(output_dir, exist_ok=True)
    try:
        shutil.copy(file_path, output_dir)
    except Exception:
        print(file_path,output_dir)

In [ ]:
files_subset_new ={}
for k,v in files_subset.items():
    files_subset_new[k]=[]
    for path in v:
        files_subset_new[k].append(f'/kaggle/input/englishdatasetbig/ASL_Citizen/videos/{path}/{k}')
files_subset_new[k][0].split(f'/{files_subset_new[k][0].split("/")[-1]}')[0]

In [ ]:
files_subset_new_test ={}
for k,v in files_subset_test.items():
    files_subset_new_test[k]=[]
    for path in v:
        files_subset_new_test[k].append(f'/kaggle/input/englishdatasetbig/ASL_Citizen/videos/{path}/{k}')
files_subset_new_test[k][0].split(f'/{files_subset_new[k][0].split("/")[-1]}')[0]

In [ ]:
def download_ucf_101_subset(zip_url, num_classes, splits, download_dir):
    for k,v in files_subset_new.items():
        for f in v:
            path = os.path.normpath(f.split(f'/{k}')[0])
            tokens = path.split(os.sep)

#     files_for_class = get_files_per_class(files)

    classes = files_subset_new.keys()

    for cls in classes:
        random.shuffle(files_subset_new[cls])

#     # Only use the number of classes you want in the dictionary
#     files_for_class = {x: files_for_class[x] for x in classes}

    dirs = {}
    for split_name, split_count in splits.items():
        print(split_name, ":")
        split_dir = Path('/kaggle/working/output') / split_name

        split_files, files_for_class = split_class_lists(files_subset_new, split_count)
        read_from_dir(zip_url, split_dir, split_files)
        dirs[split_name] = split_dir

    return dirs

In [ ]:
def download_ucf_101_subset_test(zip_url, num_classes, splits, download_dir):
    for k,v in files_subset_new_test.items():
        for f in v:
            path = os.path.normpath(f.split(f'/{k}')[0])
            tokens = path.split(os.sep)

#     files_for_class = get_files_per_class(files)

    classes = files_subset_new_test.keys()

    for cls in classes:
        random.shuffle(files_subset_new_test[cls])

#     # Only use the number of classes you want in the dictionary
#     files_for_class = {x: files_for_class[x] for x in classes}

    dirs = {}
    for split_name, split_count in splits.items():
        print(split_name, ":")
        split_dir = Path('/kaggle/working/output') / split_name

        split_files, files_for_class = split_class_lists(files_subset_new_test, split_count)
        read_from_dir(zip_url, split_dir, split_files)
        dirs[split_name] = split_dir

    return dirs

In [ ]:
# shutil.rmtree('/kaggle/working/output')

In [ ]:
download_dir = pathlib.Path('/kaggle/input/englishdatasetbig/ASL_Citizen/videos')
output = '/kaggle/working/output'
subset_paths = download_ucf_101_subset(output,
                                       num_classes = NUM_CLASSES,
                                       splits = {"train": 10, "test": 10},
                                       download_dir = download_dir)

In [ ]:
download_dir_test = pathlib.Path('/kaggle/input/englishdatasetbig/ASL_Citizen/videos')
output = '/kaggle/working/output'
subset_paths ['val'] = ( download_ucf_101_subset_test(output,
                                       num_classes = NUM_CLASSES,
                                       splits = {"val": 5},
                                       download_dir = download_dir_test))['val']

In [ ]:
# import shutil
# shutil.rmtree('/kaggle/working/output')

In [ ]:
output = pathlib.Path('/kaggle/working/output')
video_count_train = len(list(output.glob('train/*/*.mp4')))
video_count_val = len(list(output.glob('val/*/*.mp4')))
video_count_test = len(list(output.glob('test/*/*.mp4')))
print(video_count_train)
print(video_count_val)

print(video_count_test)

video_total = video_count_train + video_count_test+video_count_val
print(f"Total videos: {video_total}")

In [ ]:
subset_paths


In [ ]:
def format_frames(frame, output_size):
  """
    Pad and resize an image from a video.
    
    Args:
      frame: Image that needs to resized and padded. 
      output_size: Pixel size of the output frame image.

    Return:
      Formatted frame with padding of specified output size.
  """
  frame = tf.image.convert_image_dtype(frame, tf.float32)
  frame = tf.image.resize_with_pad(frame, *output_size)
  return frame

In [ ]:
def frames_from_video_file(video_path, n_frames, output_size = (224,224), frame_step = 10):
  """
    Creates frames from each video file present for each category.

    Args:
      video_path: File path to the video.
      n_frames: Number of frames to be created per video file.
      output_size: Pixel size of the output frame image.

    Return:
      An NumPy array of frames in the shape of (n_frames, height, width, channels).
  """
  # Read each video frame by frame
  result = []
  src = cv2.VideoCapture(str(video_path))  

  video_length = src.get(cv2.CAP_PROP_FRAME_COUNT)

  need_length = 1 + (n_frames - 1) * frame_step

  if need_length > video_length:
    start = 0
  else:
    max_start = video_length - need_length
    start = random.randint(0, max_start + 1)

  src.set(cv2.CAP_PROP_POS_FRAMES, start)
  # ret is a boolean indicating whether read was successful, frame is the image itself
  ret, frame = src.read()
  result.append(format_frames(frame, output_size))

  for _ in range(n_frames - 1):
    for _ in range(frame_step):
      ret, frame = src.read()
    if ret:
      frame = format_frames(frame, output_size)
      result.append(frame)
    else:
      result.append(np.zeros_like(result[0]))
  src.release()
  result = np.array(result)[..., [2, 1, 0]]

  return result


In [ ]:
class FrameGenerator:
  def __init__(self, path, n_frames, training = False):
    """ Returns a set of frames with their associated label. 

      Args:
        path: Video file paths.
        n_frames: Number of frames. 
        training: Boolean to determine if training dataset is being created.
    """
    self.path = path
    self.n_frames = n_frames
    self.training = training
    self.class_names = sorted(set(p.name for p in self.path.iterdir() if p.is_dir()))
    self.class_ids_for_name = dict((name, idx) for idx, name in enumerate(self.class_names))
    
  def get_files_and_class_names(self):
    video_paths = list(self.path.glob('*/*.mp4'))
    classes = [p.parent.name for p in video_paths] 
    return video_paths, classes

  def __call__(self):
    video_paths, classes = self.get_files_and_class_names()

    pairs = list(zip(video_paths, classes))

    if self.training:
      random.shuffle(pairs)
    labelmap = {}
    for path, name in pairs:
      video_frames = frames_from_video_file(path, self.n_frames) 
      label = self.class_ids_for_name[name] # Encode labels
      yield video_frames, label


In [ ]:
batch_size = 8
num_frames = 8

output_signature = (tf.TensorSpec(shape = (None, None, None, 3), dtype = tf.float32),
                    tf.TensorSpec(shape = (), dtype = tf.int16))

train_ds = tf.data.Dataset.from_generator(FrameGenerator(subset_paths['train'], num_frames, training = True),
                                          output_signature = output_signature)
train_labels = FrameGenerator(subset_paths['train'], num_frames, training = True).class_names
train_ds = train_ds.batch(batch_size)

test_ds = tf.data.Dataset.from_generator(FrameGenerator(subset_paths['test'], num_frames),
                                         output_signature = output_signature)
test_labels = FrameGenerator(subset_paths['test'], num_frames).class_names
test_ds = test_ds.batch(batch_size)

val_ds = tf.data.Dataset.from_generator(FrameGenerator(subset_paths['val'], num_frames),
                                          output_signature = output_signature)
val_labels = FrameGenerator(subset_paths['val'], num_frames).class_names
val_ds = val_ds.batch(batch_size)

In [ ]:
print(train_labels)

In [ ]:
for frames, labels in train_ds.take(4):
  print(labels)

In [ ]:
print(f"Shape: {frames.shape}")
print(f"Label: {labels.shape}")

In [ ]:
videos, labels = next(iter(train_ds))
media.show_videos(videos.numpy(), codec='gif', fps=8)
train_labels[labels[0]]

In [ ]:
videos, labels = next(iter(val_ds))
media.show_videos(videos.numpy(), codec='gif', fps=8)

In [ ]:
model_id = 'a0'
resolution = 224

tf.keras.backend.clear_session()

backbone = movinet.Movinet(model_id=model_id)
backbone.trainable = False

# Set num_classes=600 to load the pre-trained weights from the original model
model = movinet_model.MovinetClassifier(backbone=backbone, num_classes=600)
model.build([None, None, None, None, 3])

# Load pre-trained weights
!wget https://storage.googleapis.com/tf_model_garden/vision/movinet/movinet_a0_base.tar.gz -O movinet_a0_base.tar.gz -q
!tar -xvf movinet_a0_base.tar.gz

checkpoint_dir = f'movinet_{model_id}_base'
checkpoint_path = tf.train.latest_checkpoint(checkpoint_dir)
checkpoint = tf.train.Checkpoint(model=model)
status = checkpoint.restore(checkpoint_path)
status.assert_existing_objects_matched()

In [ ]:
def build_classifier(batch_size, num_frames, resolution, backbone, num_classes):
  """Builds a classifier on top of a backbone model."""
  model = movinet_model.MovinetClassifier(
      backbone=backbone,
      num_classes=num_classes)
  model.build([batch_size, num_frames, resolution, resolution, 3])

  return model

In [ ]:
batch_size, num_frames, resolution, backbone, NUM_CLASSES

In [ ]:
model = build_classifier(batch_size, num_frames, resolution, backbone, NUM_CLASSES)

In [ ]:
model.summary()


In [ ]:
num_epochs = 100
loss_obj = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

model.compile(loss=loss_obj, optimizer=optimizer, metrics=['accuracy'])

In [ ]:
train_ds, test_ds,val_ds

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  # Metric to monitor for early stopping
    mode='min',  # Set mode to 'min' for minimizing the metric
    patience=10,  # Number of epochs with no improvement before stopping
    restore_best_weights=True,  # Restore the best model weights
    verbose=1
)
rlronp=tf.keras.callbacks.ReduceLROnPlateau( monitor="val_loss", factor=0.5, patience=1,
                                             verbose=1,mode = 'min')

In [ ]:
results = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=100,
                    validation_freq=1,
                    verbose=1,batch_size=1024,callbacks=[early_stopping,rlronp])

In [ ]:
# Define the string date format.
# Get the current Date and Time in a DateTime Object.
# Convert the DateTime object to string according to the style mentioned in date_time_format string.

# Define a useful name for our model to make it easy for us while navigating through multiple saved models.
model_file_name = f'VSL_Date_Time'
os.makedirs('/kaggle/working/Model',exist_ok=True)
# Save your Model.
modelpath = f'/kaggle/working/Model/{model_file_name}'
model.save(modelpath)

In [ ]:
model_evaluation_history = model.evaluate(test_ds)

In [ ]:
model_evaluation_history = model.evaluate(train_ds)

In [ ]:
model_evaluation_history = model.evaluate(val_ds)

In [ ]:
def plot_metric(model_training_history, metric_name_1, metric_name_2, plot_name):


    # Get metric values using metric names as identifiers.
    metric_value_1 = model_training_history.history[metric_name_1]
    metric_value_2 = model_training_history.history[metric_name_2]

    # Construct a range object which will be used as x-axis (horizontal plane) of the graph.
    epochs = range(len(metric_value_1))

    # Plot the Graph.
    plt.plot(epochs, metric_value_1, 'blue', label = metric_name_1)
    plt.plot(epochs, metric_value_2, 'red', label = metric_name_2)

    # Add title to the plot.
    plt.title(str(plot_name))

    # Add legend to the plot.
    plt.legend()
# Visualize the training and validation loss metrices.
plot_metric(results, 'loss', 'val_loss', 'Total Loss vs Total Validation Loss')


In [ ]:
# Visualize the training and validation accuracy metrices.
plot_metric(results, 'accuracy', 'val_accuracy', 'Total Accuracy vs Total Validation Accuracy')

In [ ]:
k = train_ds.take(1)
labels = list(k.map(lambda x, y: y).unbatch().as_numpy_iterator())
print(labels[0])